### Algorithm 1
Extract Nouns, Noun Phrases, etc. from Dependency Tree

In [2]:
from statics import *
from pkgs import MongoClient, CoreNLPParser, defaultdict, re
client = MongoClient(DB_CONNECTION)
parser = CoreNLPParser(url=SERVER_URL)
MAX_FEATURES = 10

In [ ]:
parsed = defaultdict(list)
second_deg_unwanted_pos = set()
unwanted_pos = ['ADVP', 'ADJP', 'INTJ', 'FRAG', 'S']
second_deg_unwanted_pos.update(unwanted_pos)
for entry in client.course_evals.derivatives.ngrams.find():
    name = entry['course_id']
    print(name)
    top_features = sorted(entry['freq_to_gram_dict'].items(),key=lambda v:v[1],reverse=True)
    for f in top_features[:MAX_FEATURES]:
        parse = list(parser.parse(f[0].split()))
        s = re.findall('Tree\(([A-Z\']+)', str(parse))
        if s[1][1:-1] in second_deg_unwanted_pos: continue
        else: parsed[name].append(f)

### Save Features to DB

In [4]:
client.course_evals.derivatives.ngrams.filtered.drop()
for key in parsed.keys():
    document = {
        'course_id': key,
        'filtered_ngrams': parsed[key]
    }
    client.course_evals.derivatives.ngrams.filtered.insert_one(document)